In [19]:
import numpy as np
import matplotlib.pyplot as plt
import json
import urllib2
import datetime, time
import pandas as pd
%matplotlib inline

In [17]:
url = "https://livecache.sportresult.com/node/db/FIBASTATS_PROD/12105_GAMEACTIONS_BKM400A03_Q1_JSON.json"
response = urllib2.urlopen(url)
data = json.loads(response.read())
plays = data["content"]["full"]["Items"]
for play in plays:
    if "C1" in play and play["C1"] == "BKM400A03-99774":
        #x = time.strptime(play['Time'],'%M:%S')
        #print datetime.timedelta(hours=x.tm_hour,minutes=x.tm_min,seconds=x.tm_sec).total_seconds()
        print play

226.0
179.0
104.0
59.0


In [24]:
def check_two_pointer(json_obj):
    """
    Checks whether given json object's action is two point shot
    """
    return json_obj["AC"] == "P2"

def check_three_pointer(json_obj):
    """
    Checks whether given json object's action is three point shot
    """
    return json_obj["AC"] == "P3"

def get_all_important_feats(json_obj, quarter):
    """
    Retrieves all important data from each json object play, some that are of interest to us are:
    AC - action,
    SX - x coordinate
    SY - y coordinate
    SU - Shot result
    Time - time left in quarter
    QT : QUarter
    Result is returned as dictionary, in format of: {AC: "", SX: x, SY: y, SU: 1|0}
    """
    dict_play = {}
    dict_play["AC"] = json_obj["AC"]
    dict_play["SX"] = int(json_obj["SX"])
    dict_play["SY"] = int(json_obj["SY"])
    x = time.strptime(json_obj['Time'],'%M:%S')
    dict_play["TIME"] = datetime.timedelta(hours=x.tm_hour,minutes=x.tm_min,seconds=x.tm_sec).total_seconds()
    dict_play["QT"] = quarter
    return dict_play

In [29]:
def get_data_France():
    return [
        "https://livecache.sportresult.com/node/db/FIBASTATS_PROD/12105_GAMEACTIONS_BKM400A03_Q1_JSON.json",
        "https://livecache.sportresult.com/node/db/FIBASTATS_PROD/12105_GAMEACTIONS_BKM400A03_Q2_JSON.json",
        "https://livecache.sportresult.com/node/db/FIBASTATS_PROD/12105_GAMEACTIONS_BKM400A03_Q3_JSON.json",
        "https://livecache.sportresult.com/node/db/FIBASTATS_PROD/12105_GAMEACTIONS_BKM400A03_Q4_JSON.json",
        "https://livecache.sportresult.com/node/db/FIBASTATS_PROD/12105_GAMEACTIONS_BKM400A03_OT1_JSON.json"
    ]

def get_data_Slovenia():
    return [
        "https://livecache.sportresult.com/node/db/FIBASTATS_PROD/12105_GAMEACTIONS_BKM400A06_Q1_JSON.json",
        "https://livecache.sportresult.com/node/db/FIBASTATS_PROD/12105_GAMEACTIONS_BKM400A06_Q2_JSON.json",
        "https://livecache.sportresult.com/node/db/FIBASTATS_PROD/12105_GAMEACTIONS_BKM400A06_Q3_JSON.json",
        "https://livecache.sportresult.com/node/db/FIBASTATS_PROD/12105_GAMEACTIONS_BKM400A06_Q4_JSON.json"
    ]

def get_data_Poland():
    return [
        "https://livecache.sportresult.com/node/db/FIBASTATS_PROD/12105_GAMEACTIONS_BKM400A09_Q1_JSON.json",
        "https://livecache.sportresult.com/node/db/FIBASTATS_PROD/12105_GAMEACTIONS_BKM400A09_Q2_JSON.json",
        "https://livecache.sportresult.com/node/db/FIBASTATS_PROD/12105_GAMEACTIONS_BKM400A09_Q3_JSON.json",
        "https://livecache.sportresult.com/node/db/FIBASTATS_PROD/12105_GAMEACTIONS_BKM400A09_Q4_JSON.json",
        "https://livecache.sportresult.com/node/db/FIBASTATS_PROD/12105_GAMEACTIONS_BKM400A09_OT1_JSON.json",
        "https://livecache.sportresult.com/node/db/FIBASTATS_PROD/12105_GAMEACTIONS_BKM400A09_OT2_JSON.json"
    ]

def get_data_Greece():
    return [
        "https://livecache.sportresult.com/node/db/FIBASTATS_PROD/12105_GAMEACTIONS_BKM400A12_Q1_JSON.json",
        "https://livecache.sportresult.com/node/db/FIBASTATS_PROD/12105_GAMEACTIONS_BKM400A12_Q2_JSON.json",
        "https://livecache.sportresult.com/node/db/FIBASTATS_PROD/12105_GAMEACTIONS_BKM400A12_Q3_JSON.json",
        "https://livecache.sportresult.com/node/db/FIBASTATS_PROD/12105_GAMEACTIONS_BKM400A12_Q4_JSON.json"
    ]

def get_data_Island():
    return [
        "https://livecache.sportresult.com/node/db/FIBASTATS_PROD/12105_GAMEACTIONS_BKM400A15_Q1_JSON.json",
        "https://livecache.sportresult.com/node/db/FIBASTATS_PROD/12105_GAMEACTIONS_BKM400A15_Q2_JSON.json",
        "https://livecache.sportresult.com/node/db/FIBASTATS_PROD/12105_GAMEACTIONS_BKM400A15_Q3_JSON.json",
        "https://livecache.sportresult.com/node/db/FIBASTATS_PROD/12105_GAMEACTIONS_BKM400A15_Q4_JSON.json"
    ]

In [44]:
def get_markkanen_plays_for_game(quarter_urls, player_id):
    markkanen_plays = []
    for index, quarter_url in enumerate(quarter_urls):
        quarter = index+1
        response = urllib2.urlopen(quarter_url)
        data = json.loads(response.read())
        plays = data["content"]["full"]["Items"]
        for play in plays:
            if "C1" in play and play["C1"] == player_id:
                if check_two_pointer(play) or check_three_pointer(play):
                    markkanen_plays.append(get_all_important_feats(play, quarter))
    print len(markkanen_plays)
    return markkanen_plays

In [45]:
markkanen_plays = []
markkanen_plays.extend(get_markkanen_plays_for_game(get_data_France(), "BKM400A03-99774"))
markkanen_plays.extend(get_markkanen_plays_for_game(get_data_Slovenia(), "BKM400A06-99774"))
markkanen_plays.extend(get_markkanen_plays_for_game(get_data_Poland(), "BKM400A09-99774"))
markkanen_plays.extend(get_markkanen_plays_for_game(get_data_Greece(), "BKM400A12-99774"))
markkanen_plays.extend(get_markkanen_plays_for_game(get_data_Island(), "BKM400A15-99774"))
markkanen_data_frame = pd.DataFrame(markkanen_plays)

15
13
20
9
12


In [46]:
markkanen_data_frame

,AC,QT,SX,SY,TIME
0,P2,1,158,35,179.0
1,P3,1,235,125,59.0
2,P2,2,211,36,138.0
3,P2,2,158,56,104.0
4,P2,3,124,42,178.0
5,P2,3,157,57,171.0
6,P2,3,199,39,41.0
7,P2,4,172,55,533.0
8,P2,4,89,39,494.0
9,P2,4,181,39,212.0
